In [1]:
!python database_helpers.py

d:\Zeug\Unizeug\Master_DataScience\3.Semester\Urban Technologies\Project\database_helpers.py:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
import pymupdf
from typing import Optional
from llama_cpp import LogitsProcessorList, Llama
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.llamacpp import build_llamacpp_logits_processor
from IPython.display import display, Markdown
from llama_cpp import Llama
from lmformatenforcer import JsonSchemaParser
from huggingface_hub import hf_hub_download
from database_helpers import (
    ForeclosureCaseSchema,
    ForeclosureObjectSchema,
    ForclosureModel,
    engine,
    session
)
import os
import tempfile
import requests

In [9]:
len(session.query(ForeclosureObjectSchema).filter_by(typ = "Wohnung").filter_by(typ = "Wohnung").all())

212

In [3]:
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
"""

def get_prompt(message: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f'[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

def pdf_to_string(file_path):
    pdf_document = pymupdf.open(file_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()
    pdf_document.close()
    return text


def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

def llamacpp_with_character_level_parser(llm: Llama, prompt: str, character_level_parser: Optional[CharacterLevelParser]) -> str:
    logits_processors: Optional[LogitsProcessorList] = None
    if character_level_parser:
        logits_processors = LogitsProcessorList([build_llamacpp_logits_processor(llm, character_level_parser)])
    
    output = llm(prompt, logits_processor=logits_processors, max_tokens=1000)
    text: str = output['choices'][0]['text']
    return text

downloaded_model_path = hf_hub_download(repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF", filename="Meta-Llama-3.1-8B-Instruct-Q6_K_L.gguf") 
llm = Llama(model_path=downloaded_model_path, n_ctx=16384, n_threads=8, n_gpu_layers=-1, verbose=False, temperature=0)

In [4]:
def download_pdf(url, destination_path):
    try:
        response = requests.get(url, headers={"Referer": "https://www.zvg-portal.de/index.php?button=Suchen"}, stream=True)
        response.raise_for_status()
        if 'application/pdf' not in response.headers.get('Content-Type', ''):
            raise Exception("The URL does not point to a valid PDF file.")

        with open(destination_path, 'wb') as pdf_file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk: 
                    pdf_file.write(chunk)

        return True
    except Exception as e:
        return False

In [7]:
with open("./experiments/examples/5shot.txt", "r", encoding="UTF-8") as file:
    examples = file.read()

valid_idx = 0
all_forclosures = session.query(ForeclosureCaseSchema).all()
for idx, foreclosure_case in enumerate(all_forclosures):
    
    if session.query(ForeclosureObjectSchema).filter_by(foreclosurecase_link = foreclosure_case.link).first() or not foreclosure_case.amtliche_bekanntmachung:
        continue
    print(idx)
    with tempfile.TemporaryDirectory() as tmp_dir:
        try:
            pdf_path = os.path.join(tmp_dir, "amtliche_bekanntmachung.pdf")
            amtliche_bekanntmachung_link = f"https://www.zvg-portal.de/index.php{foreclosure_case.amtliche_bekanntmachung}"
            if not download_pdf(amtliche_bekanntmachung_link, pdf_path):
                continue
            #pdf_text = pdf_to_string(pdf_path)
            #
            #question_with_schema = (
            #    f"The following text lists examples for your task: {examples}.\n"
            #    f"You MUST answer using the following JSON schema: {ForclosureModel.model_json_schema()}.\n"
            #    f"Please extract information about the following PDF content: {pdf_text}."
            #)
            #prompt = get_prompt(question_with_schema)
            #result = llamacpp_with_character_level_parser(
            #    llm, prompt, JsonSchemaParser(ForclosureModel.model_json_schema())
            #)
#
            #if not result:
            #    continue
            #
            #foreclosure_data = ForclosureModel.model_validate_json(result.replace("\r\n", "\n"))
            #foreclosure_case.verkehrswert = foreclosure_data.gesamtverkehrswert
            #
            #for obj in foreclosure_data.objekte:
            #    obj_dict = obj.model_dump()
            #    obj_dict['raum_typen'] =",".join(obj.raum_typen)
            #    obj_data = ForeclosureObjectSchema(
            #        foreclosurecase_link = foreclosure_case.link,
            #        **obj_dict
            #    )
            #    session.add(obj_data)
            #    
            #session.commit()
            valid_idx += 1
            if valid_idx % 10 == 0:
                print(f"Processed: {idx} Valid: {valid_idx} Overall: {len(all_forclosures)}")  
                
        except Exception as e:
            print(f"Error processing foreclosure case ID {foreclosure_case.link}: {e}")
            session.rollback()

4
6
9
10
11
14
15
17
18
19
26
30
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
98
99
100
101
102
103
104
105
106
107
108
109
110
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
137
138
139
140
141
142
143
144
145
146
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
281
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
29

In [ ]:
session.close()
engine.dispose()